In [1]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xlrd

In [2]:
data1 =  pd.read_csv('data_portfolio.csv')
#print(data1)

# Extract data to variables
RetTable = data1.values# Colletively storage return of each stock in a table
stockname = data1.columns.values# obtain stock names
print(stockname)
N = len(stockname)# Number of stocks
print(N)

invstBudget = 1000    # Totol Budget of investment
expReward = 5       # Expected reward from investment

# Compute the mean of each stock and their covariance
RetMean = np.mean(RetTable, axis = 0)
print(RetMean)
RetCov = (RetTable - RetMean).T @ (RetTable - RetMean) /(RetTable.shape[0]-1)
print(RetCov)

# easy way
print(data1.mean())
RetMean = data1.mean()
print(data1.cov())
RetCov = data1.cov()

# Check wheter the covariance matrix is positive definite
Flag_posdef = np.all(np.linalg.eigvals(RetCov) > 0)
print(Flag_posdef)

# Declare Model
m = Model('portfolioMeanVar')
invest = pd.Series(m.addVars(stockname, name = 'invest'), index = stockname)

m.addConstr(quicksum(invest) <= invstBudget)
m.addConstr(quicksum(invest[i]*RetMean[i] for i in stockname) >= expReward)
m.setObjective(RetCov.dot(invest).dot(invest), GRB.MINIMIZE)
m.optimize()


# Get actual portfolio weights
p = np.array(m.getAttr('x', invest))
print("Optimal Solutions:")

# Get objective value for future usage
print("\nOptimal value: \t%g" % m.objVal)
for i in invest:
    print(" %s=  %g" %(i.varname, i.x))

ExpcRet = RetMean.dot(p)
print("Expected Reuturn = %g with investment amount %g, Risk = %g" %(ExpcRet, np.sum(p), m.objVal))






['JPM' 'AMZN' 'AAPL' 'DOC' 'GOOG']
5
[0.00481139 0.01474848 0.00513239 0.0030518  0.00539123]
[[ 5.91601443e-04 -7.33419342e-05  2.65599817e-04  8.76253303e-05
   2.28239582e-04]
 [-7.33419342e-05  2.06047471e-03  5.96828980e-04 -3.18655133e-05
   4.13388425e-04]
 [ 2.65599817e-04  5.96828980e-04  1.26006684e-03  3.58623640e-04
   5.27456999e-04]
 [ 8.76253303e-05 -3.18655133e-05  3.58623640e-04  8.24304517e-04
   1.91929451e-04]
 [ 2.28239582e-04  4.13388425e-04  5.27456999e-04  1.91929451e-04
   1.59285140e-03]]
JPM     0.004811
AMZN    0.014748
AAPL    0.005132
DOC     0.003052
GOOG    0.005391
dtype: float64
           JPM      AMZN      AAPL       DOC      GOOG
JPM   0.000592 -0.000073  0.000266  0.000088  0.000228
AMZN -0.000073  0.002060  0.000597 -0.000032  0.000413
AAPL  0.000266  0.000597  0.001260  0.000359  0.000527
DOC   0.000088 -0.000032  0.000359  0.000824  0.000192
GOOG  0.000228  0.000413  0.000527  0.000192  0.001593
True

--------------------------------------------

In [4]:
data1 =  pd.read_csv('PortfolioPriceData.csv')
#print(data1)

# Extract data to variables
data = data1.values# Colletively storage return of each stock in a table
stockname = data1.columns.values[1:6]# obtain stock names
#warning: 1:6 instead of  1:5
print(stockname)
N = len(stockname)# Number of stocks
print(N)

week = data[:,0] # Obtain week data
L = len(week) # Number of weeks
print(L)

PriceTable = data[:,1:6]
print(PriceTable.shape)
#print(PriceTable)

RetTable = np.zeros([L-1, N])
for l in range(L-1):
    for n in range(N):
        RetTable[l,n] = (PriceTable[l+1,n] - PriceTable[l,n])/PriceTable[l,n]

#print(RetTable)
RetMean = np.mean(RetTable, axis = 0)
print(RetMean)
RetCov = (RetTable - RetMean).T @ (RetTable - RetMean) /(RetTable.shape[0]-1)
print(RetCov)


Flag_posdef = np.all(np.linalg.eigvals(RetCov) > 0)
print(Flag_posdef)

invstBudget = 1;    # Totol Budget of investment
expReward = 0.002;        # Expected reward from investment
cssell = 0.2; # Maximal allowable short selling
      
# Declare Model
m = Model('portfolioMaxProb')
#x = pd.Series(m.addVars(N), index = stockname)
x = m.addVars(N, name = 'x')
z = m.addVar(name = 'z')
m.addConstr(quicksum(x) <= z)
m.addConstr(quicksum(x[i]*RetMean[i] for i in range(N)) - expReward * z == 1)
m.addConstrs(x[i] >= -cssell*z for i in range(N))

obj = quicksum(RetCov[i,j] * x[i] * x[j] for i in range(N) for j in range(N))
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()



# Get actual portfolio weights

p = np.array(m.getAttr('x', x).values())
print(p)
q = z.x

xopt = p/q
for i in range(N):
    print("Investment in stock %s = %g" %(stockname[i], xopt[i] ))

aa = RetCov.dot(xopt).dot(xopt)
bb = m.objVal
cc = (RetMean.dot(xopt) - expReward)*q
print(aa)
print(bb)
print(cc)



['GE' 'MSFT' 'JPM' 'AAPL' 'GOOG']
5
52
(52, 5)
[0.00274371 0.00120754 0.00143834 0.00163053 0.00469121]
[[0.00084264 0.00043341 0.00046712 0.00035785 0.0005861 ]
 [0.00043341 0.00111273 0.00033406 0.00038014 0.0005398 ]
 [0.00046712 0.00033406 0.00062407 0.00028221 0.0003394 ]
 [0.00035785 0.00038014 0.00028221 0.00113388 0.00049485]
 [0.0005861  0.0005398  0.0003394  0.00049485 0.00159713]]
True
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 7 rows, 6 columns and 22 nonzeros
Model fingerprint: 0x9a55ccd1
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-03, 3e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 4 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 3 quadratic objective terms
